<a href="https://colab.research.google.com/github/ElPapi42/AgeClassifier/blob/master/AgeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from IPython.display import clear_output
!pip install --upgrade tensorflow-gpu
!pip install --upgrade tqdm
!pip install --upgrade pillow
!pip install git+https://github.com/Jwink3101/parmapper
clear_output()

In [0]:
#Imports.
import PIL
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from parmapper import parmap
import sys
import pathlib

## Data Download

In [0]:
#Downloads and extract Dataset to local
#You can run this on google colab for get faster downloads speeds
import zipfile
import requests
from tqdm import tqdm

folder_path = "./Datasets"

appa_url = "http://158.109.8.102/AppaRealAge/appa-real-release.zip"
appa_path = folder_path + '/appa-real.zip'

#Create Dataset folder if not exists
if(not os.path.exists(folder_path)):
  os.mkdir(folder_path)

#download chalearn appa-real dataset
if(not(os.path.exists(appa_path) and os.path.isfile(appa_path))):
  resp = requests.get(appa_url, stream=True)

  total_size = int(resp.headers.get('content-length', 0))
  block_size = 16384
  t=tqdm(total=total_size, unit='iB', unit_scale=True)
  
  with open(appa_path, "wb") as f:
    for data in resp.iter_content(block_size):
      t.update(len(data))
      f.write(data)
    t.close()
    f.close()

    if total_size != 0 and t.n != total_size:
      print("Download Error")

  #Extract
  with zipfile.ZipFile(appa_path, 'r') as zip:
    zip.extractall(folder_path)
    zip.close()

## Exploratory Data Analysis

In [0]:
# Load Datasets
df_train = pd.read_csv("/content/Datasets/appa-real-release/gt_avg_train.csv")
df_eval = pd.read_csv("/content/Datasets/appa-real-release/gt_avg_valid.csv")
df_test = pd.read_csv("/content/Datasets/appa-real-release/gt_avg_test.csv")
df_all = pd.concat([df_train, df_eval, df_test])

In [0]:
# Lets see some examples
df_all.head()

In [0]:
# Check for nulls and other general info
df_all.info()

In [0]:
# Descriptive stats
df_all.describe()

In [0]:
# Lets see the distribution of the apparent age and the real age
sns.distplot(df_all["apparent_age_avg"], bins=100)
sns.distplot(df_all["real_age"], bins=100)

The Distribution of age dont deviate too much between the real and the apparent, we can keep it

In [0]:
# Now we will check the dist between the train, eval and test datasets
plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1, title="Apparent Avg Age")
sns.distplot(df_train["apparent_age_avg"], bins=100)
sns.distplot(df_eval["apparent_age_avg"], bins=100)
sns.distplot(df_test["apparent_age_avg"], bins=100)

plt.subplot(1, 2, 2, title="Real Age")
sns.distplot(df_train["real_age"], bins=100)
sns.distplot(df_eval["real_age"], bins=100)
sns.distplot(df_test["real_age"], bins=100)

plt.show()

This one can be better, dist of test set deviates a bit from train set, but we can keep with this for now. But this visualizations open some questions, how much the avg age deviates from the real age? putting a response here can lead us to detect some possible outliers, because the avg is based on manual reviews from real people that stimates the age of the person they see in the photo. Lets check that

In [0]:
diference = df_all["real_age"] - df_all["apparent_age_avg"]
sns.distplot(diference, bins=50)

We will ignore this deviation for now, but we can keep in mind this for the future

## Preprocessing

In [0]:
# Find URLs
df_train["file_url"] = "./Datasets/appa-real-release/train/" + df_train["file_name"] + "_face.jpg"
df_eval["file_url"] = "./Datasets/appa-real-release/valid/" + df_eval["file_name"] + "_face.jpg"
df_test["file_url"] = "./Datasets/appa-real-release/test/" + df_test["file_name"] + "_face.jpg"
df_train.head()

In [0]:
image = tf.keras.preprocessing.image.load_img("/content/Datasets/appa-real-release/test/005613.jpg_face.jpg")
image

In [0]:
#Lets see some examples
plt.figure(figsize=(10,10))
for i, path in enumerate(df_train["file_url"].iloc[:9]):
  image = tf.keras.preprocessing.image.load_img(path,)

  datapoint = df_train[df_train["file_url"] == path]
  age = datapoint["apparent_age_avg"].values[0]

  plt.subplot(3,3,i+1, title="age: {age}".format(age=age))
  plt.imshow(image)
plt.show()

## Dataset Creation

In [0]:
# Generator funcion
def load_image(path, label):
  image = tf.io.read_file(path)
  image = tf.image.decode_image(image)
  image = tf.image.resize_with_pad(image, 256, 256, antialias=True)
  return image, label

def transform_image(image_batch, label_batch):
  image_batch = image_batch / 255.0
  return image_batch, label_batch

In [0]:
# Hyper Parameters
batch_size = 16

In [0]:
#Create Datasets for train, evaluation and testing
train_ds = tf.data.Dataset.from_tensor_slices((df_train["file_url"], df_train["apparent_age_avg"]))
train_ds = train_ds.map(load_image, tf.data.experimental.AUTOTUNE)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.map(transform_image, tf.data.experimental.AUTOTUNE)
train_df = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

#eval_ds = tf.data.Dataset.from_generator(image_loader, output_types=tf.float32, args=[df_eval["file_url"]])
#test_ds = tf.data.Dataset.from_generator(image_loader, output_types=tf.float32, args=[df_test["file_url"]])

In [0]:
for img in train_ds.take(100):
  pass